In [29]:
import pandas as pd
import datetime

In [39]:
# Read ismr file 
ismr_fileName = "ljic246a30.20_.ismr"
data = pd.read_csv(ismr_fileName, usecols=[7,8,32,33], header=None, squeeze=True)
data

,7,8,32,33
0,0.033,0.032,0.048,0.045
1,0.180,0.110,nan,nan
2,0.120,0.055,0.134,0.062
3,0.033,0.036,nan,nan
4,0.057,0.053,nan,nan
...,...,...,...,...
499,0.055,0.038,nan,nan
500,0.029,0.027,nan,nan
501,0.038,0.028,nan,nan
502,0.154,0.118,nan,nan


In [22]:
#
WN = data[0] # Week number, since first epoch 
TOW = data[1] # Time of week (s)
PRN = data[2]
S4 = data[7] # S4 in "Sig1"
alpha = data[4]
epsilon = data[5]
#data
PRN

0       10
1       12
2       18
3       20
4       21
      ... 
492    162
493    164
494    166
495    174
496    175
Name: 2, Length: 497, dtype: int64

In [3]:
a = list(data.iloc[0,[0,1]])
a

[2117.0, 350160.0]

In [4]:
# GPS time: week & seconds; to UTC time.
# OJO: It's missing the leapseconds, these value get from the navigation file
def WeekSeconds2UTC(gpsweek,gpsseconds,leapseconds):
    import datetime
    
    datetimeformat = "%Y-%m-%d %H:%M:%S"
    first_epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
    elapsed = datetime.timedelta(days=(gpsweek*7),seconds=(gpsseconds-leapseconds))
    
    # LISN date format: 2 digit year, day of year, seconds since midnight for a day 
    date = datetime.datetime.strftime(first_epoch + elapsed,"%y-%j;%H:%M:%S")
    
    year = int(date[:2])
    day_year = int(date[3:6])
    hour = int(date[7:9])
    minute = int(date[10:12])
    second = int(date[13:15])
    seconds_day = second*1 + minute*60 + hour*60*60
    
    # Get current date for LISN file names
    date_name = datetime.datetime.strftime(first_epoch + elapsed,"%y-%m-%d")
    
    year_n = date_name[:2]
    month_n = date_name[3:5]
    day_n = date_name[6:8]
    
    return {"year":year, "day_year":day_year, "seconds_day": seconds_day, "year_name":year_n, "month_name":month_n, "day_name": day_n}
    

In [5]:
a = WeekSeconds2UTC(2109,342000,0)
a["year_name"], a["month_name"], a["day_name"]
    

('20', '06', '10')

In [6]:
# The PRNs code were obtained from PolaRx5S Reference Guide
def get_PRN(svid):
    if 1<=svid<=37:
        prn = "G"+str(svid)
    elif 38<=svid<=61:
        prn = "R"+str(svid-37)
    elif svid==62:
        prn = "NA"
    elif 63<=svid<=68:
        prn = "R"+str(svid-38)
    elif 71<=svid<=106:
        prn = "E"+str(svid-70)
    elif 107<=svid<=119:
        prn = "NA"
    elif 120<=svid<=140:
        prn = "S"+str(svid-100)
    elif 141<=svid<=177:
        prn = "C"+str(svid-140)
    elif 181<=svid<=187:
        prn = "J"+str(svid-180)
    elif 191<=svid<=197:
        prn = "I"+str(svid-190)
    elif 198<=svid<=215:
        prn = "S"+str(svid-157)
    elif 216<=svid<=222:
        prn = "I"+str(svid-208)
    else:
        prn = "svid not valid!"
    
    return prn

In [7]:
# Create new columns of lisn file 
yr = [] # year
dyr = [] # day_year
sd = [] # seconds_day
prn = [] # prn
#
for i in range(len(WN)):
    fecha = WeekSeconds2UTC(int(WN[i]), int(TOW[i]), 0)
    yr.append(fecha["year"])
    dyr.append(fecha["day_year"])
    sd.append(fecha["seconds_day"])   
    prn.append(get_PRN(int(PRN[i])))

In [8]:
# Create the lisn dataframe
list_tuples = list(zip(yr,dyr,sd,prn)) # creates a list of tuples
dataFrame = pd.DataFrame(list_tuples, columns=['year','day','seconds','PRN']) # creates a pandas dataframe

# Adding other pandas series
dataFrame['S4'] = S4
dataFrame['alpha'] = alpha
dataFrame['epsilon'] = epsilon

dataFrame

,year,day,seconds,PRN,S4,alpha,epsilon
0,20,219,4560,G10,0.097,3.0,26.0
1,20,219,4560,G12,0.147,61.0,14.0
2,20,219,4560,G18,0.043,169.0,64.0
3,20,219,4560,G20,0.040,38.0,49.0
4,20,219,4560,G21,0.045,304.0,36.0
...,...,...,...,...,...,...,...
492,20,219,5400,C22,0.105,46.0,28.0
493,20,219,5400,C24,0.107,223.0,27.0
494,20,219,5400,C26,0.025,269.0,71.0
495,20,219,5400,C34,0.163,274.0,11.0


In [9]:
def create_row(seconds):
    # Select a subset of data 
    subset1 = dataFrame[dataFrame["seconds"] == seconds]
    # Create the list header
    header = list(subset1.iloc[0,:3])
    header.append(len(subset1))
    # Create the list body
    ss1 = subset1.loc[:,"PRN":] # Select some columns
    # Merge the list header and body 
    for i in range(len(ss1)):
        header += list(ss1.iloc[i])
    # Create the row dataframe 
    parameters = pd.DataFrame([header])
    
    return parameters 

In [10]:
seconds_list = dataFrame['seconds']
seconds_list = list(seconds_list.drop_duplicates())
seconds_list

[4560,
 4620,
 4680,
 4740,
 4800,
 4860,
 4920,
 4980,
 5040,
 5100,
 5160,
 5220,
 5280,
 5340,
 5400]

In [11]:
for i in range(len(seconds_list)):
    if i == 0: 
        result = create_row(seconds_list[i])
    else:
        result = result.append(create_row(seconds_list[i]),ignore_index=True)
result = result.round(3) # round decimal numerical values to 3 numbers
result

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,20,219,4560,33,G10,0.097,3.0,26.0,G12,0.147,...,280.0,9.0,C35,0.095,165.0,34.0,NaN,NaN,NaN,NaN
1,20,219,4620,33,G10,0.108,3.0,26.0,G12,0.240,...,280.0,9.0,C35,0.116,165.0,33.0,NaN,NaN,NaN,NaN
2,20,219,4680,33,G10,0.112,3.0,27.0,G12,0.164,...,279.0,9.0,C35,0.148,164.0,33.0,NaN,NaN,NaN,NaN
3,20,219,4740,33,G10,0.102,3.0,27.0,G12,0.157,...,279.0,9.0,C35,0.117,164.0,33.0,NaN,NaN,NaN,NaN
4,20,219,4800,33,G10,0.110,4.0,27.0,G12,0.120,...,278.0,10.0,C35,0.117,163.0,33.0,NaN,NaN,NaN,NaN
5,20,219,4860,33,G10,0.141,4.0,28.0,G12,0.134,...,278.0,10.0,C35,0.128,163.0,32.0,NaN,NaN,NaN,NaN
6,20,219,4920,33,G10,0.117,4.0,28.0,G12,0.143,...,278.0,10.0,C35,0.078,163.0,32.0,NaN,NaN,NaN,NaN
7,20,219,4980,34,G10,0.114,4.0,28.0,G12,0.162,...,259.0,71.0,C34,0.115,277.0,10.0,C35,0.073,162.0,32.0
8,20,219,5040,34,G10,0.104,5.0,29.0,G12,0.136,...,260.0,71.0,C34,0.114,277.0,10.0,C35,0.093,162.0,32.0
9,20,219,5100,33,G10,0.088,5.0,29.0,G12,0.150,...,276.0,10.0,C35,0.097,162.0,32.0,NaN,NaN,NaN,NaN


In [12]:
a = list(result.iloc[0,[0,1]])
today = str(a[0])+"-"+str(a[1])+";00:00:00"
fecha = datetime.datetime.strptime(today,"%y-%j;%H:%M:%S")
fecha = datetime.datetime.strftime(fecha,"%y-%m-%d")
fecha[6:8]
#today

'06'

In [13]:
station_name = ismr_fileName[:4]
station_name

'ljic'

In [14]:
def get_file_name():
    station_name = ismr_fileName[:4]
    a = list(data.iloc[0,[0,1]]) # Get WN and TOW for the first row
    current_time = WeekSeconds2UTC(a[0],a[1],0)
    year = current_time["year_name"]
    month = current_time["month_name"]
    day = current_time["day_name"]
    
    file_name = station_name + "_" + year + month + day + ".s4"
    
    return file_name

In [15]:
s4_name = get_file_name()
result.to_csv(s4_name, sep='\t',index=False,header=False,encoding='utf-8') # sep='\b'

In [16]:
m = list(result[5])
m[5]

0.141